# Flat Earth Experiment

#### Torsten Auerswald (NCAS, University of Reading)

## Motivation
In this experiment, you will be using SpeedyWeather to
investigate the influence of continents, and especially large mountains, on the global circulation of
the atmosphere. You will use this model to investigate the role of mountains for the general
circulation and climate on Earth.

## Experiment
This notebook is for running SpeedyWeather with and without orography. To run the control run comment this line out:

`orography=NoOrography(spectral_grid)`

Conduct both simulations (control run and flat earth experiment) and remember to change the run id in each simulation. After the simulations have finished you can use the notebook "Flat_Earth_analysis.ipynb" to have a first look at the data and get some ideas for further analysis.

## Guiding questions
 - How does the midlatitude circulation change in the flat earth experiment? Have a look at surface pressure, and geopotential height and winds at different levels.
 - What is the role of topography over East Africa and the Tibetan Plateau?
 - Do the meridional atmospheric transports of heat and moisture change?

## Further Reading
For more details on the theoretical background and literature have a look at the lab notes for this experiment in "Flat_Earth_Labnotes.pdf"

In [3]:
using Pkg

In [4]:
Pkg.status("SpeedyWeather")

Status `~/.julia/environments/v1.11/Project.toml`
  [9e226e20] SpeedyWeather v0.16.0 `~/.julia/dev/SpeedyWeather`


In [ ]:
#Use this only to update to a new version of SpeedyWeather
#Main release
#Pkg.add("SpeedyWeather")
#Developer version
#Pkg.develop("SpeedyWeather")

In [ ]:
using SpeedyWeather
# Define grid
spectral_grid = SpectralGrid(trunc=31, nlayers=8)
# Define output. Change the id for the control run and the flat earth run
output = NetCDFOutput(spectral_grid, PrimitiveWetModel, path="./", id="Flat_Earth", output_dt=Minute(360))



In [ ]:
# Some parameterisation choices
vertical_advection = WENOVerticalAdvection(spectral_grid)

#Define albedo
albedo = Albedo(ocean=OceanSeaIceAlbedo(spectral_grid), land=AlbedoClimatology(spectral_grid))


In [ ]:
# Remove orography
# Comment this out to run with orography
orography=NoOrography(spectral_grid)

In [ ]:
### Set up the model to run from scratch
model = PrimitiveWetModel(spectral_grid; orography,vertical_advection, albedo, output)

In [ ]:
simulation = initialize!(model)
# now offset via set! with add=true
set!(albedo.land, (λ, φ) -> 0.1, add=true)

# Add output variables
add!(model, SpeedyWeather.OceanOutput()...)
add!(model, SpeedyWeather.LandOutput()...)
add!(model, SpeedyWeather.RadiationOutput()...)
add!(model, SpeedyWeather.PrecipitationOutput()...)
add!(model, SpeedyWeather.SurfaceFluxesOutput()...)
#This adds orography variable to output
add!(model, SpeedyWeather.BoundaryOutput()...)



In [ ]:
#Check model settings
model

In [ ]:
#Plot orography to make sure we set it to zero
using CairoMakie
heatmap(orography.orography)

In [12]:
run!(simulation, period=Year(30), output=true)

Weather is speedy: run_Flat_Earth_SeaIce_0001 100% Time: 1:07:24 (640.86 years/day)


Simulation{PrimitiveWetModel}
├ prognostic_variables::PrognosticVariables{...}
├ diagnostic_variables::DiagnosticVariables{...}
└ model::PrimitiveWetModel{...}